In [1]:
!pip install striprtf

In [2]:
import striprtf
from striprtf.striprtf import rtf_to_text

import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('punkt_tab')

import matplotlib.pyplot as plt
import numpy as np
import re, string
from collections import Counter

import random

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import math
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from collections import Counter

In [4]:
def load_convert(file_path):
    """Loads an RTF file, converts it to plain text"""
    try:
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
            rtf_content = f.read()

        # Convert RTF to plain text
        plain_text = rtf_to_text(rtf_content)
        
        return plain_text

    except FileNotFoundError:
        print(f"Error: File not found at {file_path}. Returning empty string.")
        return ""
    except Exception as e:
        print(f"An error occurred while processing {file_path}: {e}")
        return ""

In [5]:
file_path_1 = '/kaggle/input/dune-frank-herbert/dune/dune.rtf'
file_path_2 = '/kaggle/input/dune-frank-herbert/dune/dune-messiah.rtfd/TXT.rtf'

text_dune = load_convert(file_path_1)
text_dune_messiah = load_convert(file_path_2)

separator = "\n\n--- END OF DUNE / START OF DUNE MESSIAH ---\n\n"
combined_text = text_dune + separator + text_dune_messiah

In [6]:
combined_text[:500]

'Книга перша\u2028Dюна\n1\nПочаток — то мить, коли варто якнайкраще подбати про істинну рівновагу речей. Кожна сестра Бене Ґессерит[2] знає про це. Тож, приступаючи до вивчення життя Муад’Діба, зважте спершу на те, у які часи він з’явився, бо ж народився владар у рік п’ятдесят сьомий правління Падишаха-Імператора Шаддама IV. Й особливо уважно придивіться, де саме він з’явився: на планеті Арракіс. Хай не вводить вас в оману той факт, що народився він і прожив перші п’ятнадцять років свого життя на Калада'

In [7]:
def clean_dune_text(text):

    # Convert to lowercase
    text = text.lower()
    
    # Remove citation markers like [2], [10], etc.
    text = re.sub(r'\[\d+\]', '', text)

    # Remove numbers (digits 0-9)
    text = re.sub(r'\d+', '', text)

    # Fix specific OCR/Typo: Latin 'D' to Cyrillic 'Д' in 'Dюна'
    text = text.replace('dюна', 'дюна')

    # Remove punctuation
    #text = re.sub(r'[^\w\s]', '', text)

    # Normalize whitespace:
    text = re.sub(r'\s+', ' ', text)

    # Strip leading/trailing whitespace
    return text.strip()

cleaned_text = clean_dune_text(combined_text)
print(cleaned_text[:500])

книга перша дюна початок — то мить, коли варто якнайкраще подбати про істинну рівновагу речей. кожна сестра бене ґессерит знає про це. тож, приступаючи до вивчення життя муад’діба, зважте спершу на те, у які часи він з’явився, бо ж народився владар у рік п’ятдесят сьомий правління падишаха-імператора шаддама iv. й особливо уважно придивіться, де саме він з’явився: на планеті арракіс. хай не вводить вас в оману той факт, що народився він і прожив перші п’ятнадцять років свого життя на каладані. а


Tokenize the text into words

In [8]:
words = cleaned_text.split()
word_counts = Counter(words)
vocab = list(word_counts.keys())
vocab_size = len(vocab)

word_to_int = {word: i for i, word in enumerate(vocab)}
int_to_word = {i: word for word, i in word_to_int.items()}
SEQUENCE_LENGTH = 64
samples = [words[i:i+SEQUENCE_LENGTH+1] for i in range(len(words)-SEQUENCE_LENGTH)]

print(vocab[:50])
print(dict(list(word_to_int.items())[:50]))
print(dict(list(int_to_word.items())[:50]))

['книга', 'перша', 'дюна', 'початок', '—', 'то', 'мить,', 'коли', 'варто', 'якнайкраще', 'подбати', 'про', 'істинну', 'рівновагу', 'речей.', 'кожна', 'сестра', 'бене', 'ґессерит', 'знає', 'це.', 'тож,', 'приступаючи', 'до', 'вивчення', 'життя', 'муад’діба,', 'зважте', 'спершу', 'на', 'те,', 'у', 'які', 'часи', 'він', 'з’явився,', 'бо', 'ж', 'народився', 'владар', 'рік', 'п’ятдесят', 'сьомий', 'правління', 'падишаха-імператора', 'шаддама', 'iv.', 'й', 'особливо', 'уважно']
{'книга': 0, 'перша': 1, 'дюна': 2, 'початок': 3, '—': 4, 'то': 5, 'мить,': 6, 'коли': 7, 'варто': 8, 'якнайкраще': 9, 'подбати': 10, 'про': 11, 'істинну': 12, 'рівновагу': 13, 'речей.': 14, 'кожна': 15, 'сестра': 16, 'бене': 17, 'ґессерит': 18, 'знає': 19, 'це.': 20, 'тож,': 21, 'приступаючи': 22, 'до': 23, 'вивчення': 24, 'життя': 25, 'муад’діба,': 26, 'зважте': 27, 'спершу': 28, 'на': 29, 'те,': 30, 'у': 31, 'які': 32, 'часи': 33, 'він': 34, 'з’явився,': 35, 'бо': 36, 'ж': 37, 'народився': 38, 'владар': 39, 'рік': 

# Dataset

In [9]:
class TextDataset(Dataset):
    def __init__(self, samples, word_to_int):
        self.samples = samples
        self.word_to_int = word_to_int
    def __len__(self):
        return len(self.samples)
    def __getitem__(self, idx):
        sample = self.samples[idx]
        input_seq = torch.LongTensor([self.word_to_int[word] for word in sample[:-1]])
        target_seq = torch.LongTensor([self.word_to_int[word] for word in sample[1:]])
        return input_seq, target_seq

In [10]:
BATCH_SIZE = 32
dataset = TextDataset(samples, word_to_int)
dataloader = DataLoader(
    dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
)
print(dataset[1])

(tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 11, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 31, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,
        34, 53, 29, 54, 55, 56, 57, 58, 59, 60]), tensor([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
        11, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
        37, 38, 39, 31, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 34,
        53, 29, 54, 55, 56, 57, 58, 59, 60, 61]))


# Model (Decoder Only Text Generation Transformer)

In [11]:
def generate_square_subsequent_mask(sz):
    """"
    Generate a square mask for the sequence. The masked positions are filled with float('-inf').
    Unmasked positions are filled with float(0.0).
    """
    mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

In [12]:
class PositionalEncoding(nn.Module):
    def __init__(self, max_len, d_model, dropout=0.1):
        """
        :param max_len: Input length sequence.
        :param d_model: Embedding dimension.
        :param dropout: Dropout value (default=0.1)
        """
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        """
        Inputs of forward function
        :param x: the sequence fed to the positional encoder model (required).
        Shape:
            x: [sequence length, batch size, embed dim]
            output: [sequence length, batch size, embed dim]
        """
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)

In [13]:
class TextGen(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_layers, num_heads):
        super(TextGen, self).__init__()
        self.pos_encoder = PositionalEncoding(max_len=SEQUENCE_LENGTH, d_model=embed_dim)
        self.emb = nn.Embedding(vocab_size, embed_dim)
        self.decoder_layer = nn.TransformerDecoderLayer(
            d_model=embed_dim, 
            nhead=num_heads, 
            batch_first=True
        )
        self.decoder = nn.TransformerDecoder(
            decoder_layer=self.decoder_layer,
            num_layers=num_layers,
        )
        self.linear = nn.Linear(embed_dim, vocab_size)
        self.dropout = nn.Dropout(0.2)
        
    # Positional encoding is required. Else the model does not learn.
    def forward(self, x):
        emb = self.emb(x)
        
        # Generate input sequence mask with shape (SEQUENCE_LENGTH, SEQUENCE_LENGTH)
        input_mask = generate_square_subsequent_mask(x.size(1)).to(x.device)
        
        x = self.pos_encoder(emb)
        x = self.decoder(x, memory=x, tgt_mask=input_mask, memory_mask=input_mask)
        x = self.dropout(x)
        out = self.linear(x)
        return out

# Training

In [14]:
def train(model, epochs, dataloader, criterion):
    model.train()
    for epoch in range(epochs):
        running_loss = 0
        for input_seq, target_seq in dataloader:
            input_seq, target_seq = input_seq.to(device), target_seq.to(device)
            outputs = model(input_seq)
            target_seq = target_seq.contiguous().view(-1)
            outputs = outputs.view(-1, vocab_size)
            
            loss = criterion(outputs, target_seq.view(-1))
    
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.detach().cpu().numpy()
        epoch_loss = running_loss / len(dataloader)
        print(f"Epoch {epoch} loss: {epoch_loss:.3f}")

In [15]:
epochs = 100
learning_rate = 0.001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = TextGen(
    vocab_size=vocab_size, 
    embed_dim=100,
    num_layers=2, 
    num_heads=2,
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

print(model)

total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")

total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.\n")

TextGen(
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (emb): Embedding(51186, 100)
  (decoder_layer): TransformerDecoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=100, out_features=100, bias=True)
    )
    (multihead_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=100, out_features=100, bias=True)
    )
    (linear1): Linear(in_features=100, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=100, bias=True)
    (norm1): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
    (norm3): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
    (dropout3): Dropout(p=0.1, inplace=False)
  )
  (decoder): Transfor

In [16]:
train(model, epochs, dataloader, criterion) 

Epoch 0 loss: 4.550
Epoch 1 loss: 2.665
Epoch 2 loss: 2.052
Epoch 3 loss: 1.631
Epoch 4 loss: 1.321
Epoch 5 loss: 1.094
Epoch 6 loss: 0.929
Epoch 7 loss: 0.808
Epoch 8 loss: 0.718
Epoch 9 loss: 0.651
Epoch 10 loss: 0.598
Epoch 11 loss: 0.556
Epoch 12 loss: 0.522
Epoch 13 loss: 0.494
Epoch 14 loss: 0.471
Epoch 15 loss: 0.450
Epoch 16 loss: 0.433
Epoch 17 loss: 0.418
Epoch 18 loss: 0.405
Epoch 19 loss: 0.393
Epoch 20 loss: 0.383
Epoch 21 loss: 0.374
Epoch 22 loss: 0.365
Epoch 23 loss: 0.357
Epoch 24 loss: 0.351
Epoch 25 loss: 0.345
Epoch 26 loss: 0.339
Epoch 27 loss: 0.333
Epoch 28 loss: 0.329
Epoch 29 loss: 0.324
Epoch 30 loss: 0.320
Epoch 31 loss: 0.316
Epoch 32 loss: 0.313
Epoch 33 loss: 0.309
Epoch 34 loss: 0.306
Epoch 35 loss: 0.303
Epoch 36 loss: 0.300
Epoch 37 loss: 0.297
Epoch 38 loss: 0.295
Epoch 39 loss: 0.292
Epoch 40 loss: 0.290
Epoch 41 loss: 0.288
Epoch 42 loss: 0.286
Epoch 43 loss: 0.284
Epoch 44 loss: 0.282
Epoch 45 loss: 0.280
Epoch 46 loss: 0.279
Epoch 47 loss: 0.277
Ep

# Inference

In [17]:
def return_int_vector(text):
    words = text.split()
    input_seq = torch.LongTensor([word_to_int[word] for word in words[-SEQUENCE_LENGTH:]]).unsqueeze(0)
    return input_seq
    
def sample_next(predictions):
    """
    Greedy sampling.
    """
    # Greedy approach.
    probabilities = F.softmax(predictions[:, -1, :], dim=-1).cpu()
    next_token = torch.argmax(probabilities)
    return int(next_token.cpu())
    
def text_generator(sentence, generate_length):
    model.eval()
    sample = sentence
    for i in range(generate_length):
        int_vector = return_int_vector(sample)
        if len(int_vector) >= SEQUENCE_LENGTH - 1:
            break
        input_tensor = int_vector.to(device)
        with torch.no_grad():
            predictions = model(input_tensor)
        next_token = sample_next(predictions)
        sample += ' ' + int_to_word[next_token]
    print(sample)
    print('\n')

In [18]:
sentences = [
    "імператор пол був"
]

generate_length = 100
for sentence in sentences:
    print(f"PROMPT: {sentence}")
    text_generator(sentence, generate_length)

PROMPT: імператор пол був
імператор пол був безумцем, дункане! — не кажи так! — різко відповів він. — увесь всесвіт скаже це, перш ніж я договорю. — чого б це, заради любові до небес? — заради любові до мого брата, не до небес. дзен-сунітська проникливість розширила його свідомість. він бачив, що алія не мала видінь — жодного, відколи померла чані. — дивна твоя любов, — сказав він. — любов! дункане, йому достатньо було вийти з колії! яке значення мала решта всесвіту, що завалилася б позаду нього? він був би в безпеці… і чані з ним! — то… чому ж не зробив так? — заради любові до небес,


